# Get audio features and genres from Spotify

### Standard Import

In [0]:
# set up all that malarkey
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd

### Personal Functions

In [0]:
def import_data(name='spotify_data.csv'):
  file_downloaded = drive.CreateFile({'id': '13uzFEoT3E4TAr6zLfDsyIYn4yUmRlgkZ'}) #change the file id with new files
  file_downloaded.GetContentFile(name)
  df = pd.read_csv(name)
  # remove empty rows
  df.dropna(inplace=True)
  # convert string to numbers
  df.Position = pd.to_numeric(df.Position, errors='coerce')
  df.Streams = pd.to_numeric(df.Streams, errors='coerce')
  df.Date = pd.to_datetime(df.Date, errors='coerce')
  # fill empty
  df.Streams = df.Streams.fillna(0)
  df.Position = df.Position.fillna(0)
  # todo - c'è una riga che ha URL come track id e che sputtana tutto
  return df

### Load the CSV file directly from Google Drive into a Dataframe

In [0]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

df=import_data()

## Load the csv file directly from Google Drive into a Dataframe

In [ ]:
df.head()

### Setup spotify

In [ ]:
# import non recursive spotipy
!pip install spotipy

# import needed libraries
import time
import numpy as np
import json
import io
import spotipy
import config.py
from spotipy.oauth2 import SpotifyClientCredentials

In [0]:
# connect to spotify
client_id = client_id # change with your keys
client_secret = client_secret # change with your keys

client_credential = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credential)

## Preparing the dataset and getting the audio features per each track

About getting the audio features: https://developer.spotify.com/web-api/get-audio-features/

We will look first for the unique songs and loop trought their IDs to get the audio features.
(Hint: on 84000 rows I have 2014 unique songs... Looping for each song for each day it would have take me days)

Because there could be some errors, the saved items will be stored in a different dataframe and the remaiining will be assigned to the same dataframe and looped again.

In [10]:
# number of unique tracks
df_track = df['Track_Id'].drop_duplicates().reset_index()
df_track.shape[0]

417

In [ ]:
# create new empty column to store the audio features from spotify
new_columns = ['artist_id','track_genres','acousticness','danceability','duration_ms','energy','instrumentalness','liveness','loudness','speechiness']
df_track = df_track.reindex(columns=[*df.columns.tolist(), *new_columns], fill_value=0.0)
# dunno why I have those columns again
df_track.drop(['0','Position','Track_Name','Artist','Streams','Date'],axis=1, inplace=True)
# do a preview
df_track.head()

In [0]:
# to test, reduce the df a few rows
#df_track = df_track.head(1)

In [0]:
# I really need a more elegant way to do this, as this is so slow!

# note: about SettingCopyWithWarning
# https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
pd.options.mode.chained_assignment = None #hide warning
for i, row in df_track.iterrows():
    audio_ft = sp.audio_features(df_track['Track_Id'][i])
    song_data = sp.track(df_track['Track_Id'][i])['artists'][0]['id']        
    # turns the list badly shape as a dictionary
    audio_ft = audio_ft[0]
    # store data into the dataframe
    df_track['artist_id'][i] = song_data
    df_track['acousticness'][i] = audio_ft['acousticness']
    df_track['danceability'][i] = audio_ft['danceability']
    df_track['duration_ms'][i] = audio_ft['duration_ms']
    df_track['energy'][i] = audio_ft['energy']
    df_track['instrumentalness'][i] = audio_ft['instrumentalness']
    df_track['liveness'][i] = audio_ft['liveness']
    df_track['loudness'][i] = audio_ft['loudness']
    df_track['speechiness'][i] = audio_ft['speechiness']
    df_track['track_genres'][i] = sp.artist(df_track['artist_id'][i])['genres']
    #df_track['artist_image'][i] = sp.artist(df_track['artist_id'][i])['artist_image'] # THIS NEED TO BE TESTED
    time.sleep(0.2) #this could take a long time

In [13]:
#some check to find instances with no data
df_track[df_track['energy'] == 0].shape[0]

0

## Merge and clean and save

At this point, I should merge back the data with the original chart and save it somewhere for further analysis

In [14]:
#join the files

df_song_data = df.merge(df_track, how='inner', on=['Track_Id'])
df_song_data.head(5)

,0,Position,Track_Name,Artist,Streams,Track_Id,Date,artist_id,track_genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness
0,2,1,Cupido (feat. Quavo),Sfera Ebbasta,364233,0dLToDou8YHYJpQltj3Ijy,2018-02-23,23TFHmajVfBtlRx5MXqgoz,[italian hip hop],0.0622,0.803,210107.0,0.695,0.0,0.105,-5.164,0.0413
1,36,35,Cupido (feat. Quavo),Sfera Ebbasta,114034,0dLToDou8YHYJpQltj3Ijy,2018-07-24,23TFHmajVfBtlRx5MXqgoz,[italian hip hop],0.0622,0.803,210107.0,0.695,0.0,0.105,-5.164,0.0413
2,42,41,Cupido (feat. Quavo),Sfera Ebbasta,82289,0dLToDou8YHYJpQltj3Ijy,2018-08-25,23TFHmajVfBtlRx5MXqgoz,[italian hip hop],0.0622,0.803,210107.0,0.695,0.0,0.105,-5.164,0.0413
3,43,42,Cupido (feat. Quavo),Sfera Ebbasta,74777,0dLToDou8YHYJpQltj3Ijy,2018-08-26,23TFHmajVfBtlRx5MXqgoz,[italian hip hop],0.0622,0.803,210107.0,0.695,0.0,0.105,-5.164,0.0413
4,44,43,Cupido (feat. Quavo),Sfera Ebbasta,78636,0dLToDou8YHYJpQltj3Ijy,2018-08-27,23TFHmajVfBtlRx5MXqgoz,[italian hip hop],0.0622,0.803,210107.0,0.695,0.0,0.105,-5.164,0.0413


In [0]:
df_song_data.drop('0', axis=1, inplace=True)

In [ ]:
# convert string to numbers
df_song_data.Position = df_song_data.to_numeric(df.Position, errors='coerce')
df_song_data.Streams = df_song_data.to_numeric(df.Streams, errors='coerce')
df_song_data.Date = df_song_data.to_datetime(df.Date, errors='coerce')
#df.drop('Track_Id', axis=1, inplace=True)
#df.drop('artist_id', axis=1, inplace=True)

# clean track genres, this is up to you if you want to clean the json format
df_song_data['track_genres'] = df_song_data['track_genres'].str.replace("[","")
df_song_data['track_genres'] = df_song_data['track_genres'].str.replace("]","")
df_song_data['track_genres'] = df_song_data['track_genres'].str.replace("'","")
df_song_data.info()

In [18]:
df_song_data.tail(5)

,Position,Track_Name,Artist,Streams,Track_Id,Date,artist_id,track_genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness
1795,75,Nice Guy (with Jessie Reyez),Eminem,59840,09FfQO3PqHO4HIJL511Ksi,2018-08-31,7dGJo4pcD2V6oG8kP0tJRR,"[detroit hip hop, g funk, hip hop, rap]",0.0942,0.650,150802.0,0.639,0.0,0.1350,-5.962,0.4580
1796,79,Em Calls Paul - Skit,Eminem,58742,6B3zy3LOKHndqsviCr2z15,2018-08-31,7dGJo4pcD2V6oG8kP0tJRR,"[detroit hip hop, g funk, hip hop, rap]",0.9770,0.707,49024.0,0.314,0.0,0.1090,-10.115,0.7470
1797,87,Good Guy (feat. Jessie Reyez),Eminem,56448,4g32MdRoqwGKQd3NXIRhpU,2018-08-31,7dGJo4pcD2V6oG8kP0tJRR,"[detroit hip hop, g funk, hip hop, rap]",0.2440,0.696,142192.0,0.634,0.0,0.4370,-6.918,0.2810
1798,156,BEBE,6ix9ine,34229,7ATATJztoWLcqDAcVVof0C,2018-08-31,7gZfnEnfiaHzxARJ2LeXrf,[emo rap],0.1720,0.635,217822.0,0.902,0.0,0.1250,-3.559,0.1090
1799,183,Lucky Strike,Troye Sivan,29389,5mTJcvxRxq3uF5wUjjciDw,2018-08-31,3WGpXCj9YhhfX11TToZcXP,"[dance pop, electropop, pop, post-teen pop, vi...",0.5790,0.612,208967.0,0.694,0.0,0.0914,-7.363,0.0514


In [0]:
#prompt download
#to-do: save directly to drive and return file id
df_song_data.to_csv("spotify_full_data.csv", index=False)

In [0]:
from google.colab import files
files.download('spotify_full_data.csv')